In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

In [8]:
import jupyterlab_dash
viewer = jupyterlab_dash.AppViewer()

In [9]:
daily = pd.read_csv('https://raw.githubusercontent.com/nithvijay/crime-rates/master/daily.csv')
#weekly = pd.read_csv('')

,Date,Close,Num Arrests,Mean.TemperatureF,PrecipitationIn,Events,season,Year,Week,Month
0,2006-01-03,1268.800049,899,38.0,0.98,Rain,Winter,2006,1,1
1,2006-01-04,1273.459961,1229,34.0,T,NaN,Winter,2006,1,1
2,2006-01-05,1273.479980,1383,43.0,0.04,NaN,Winter,2006,1,1
3,2006-01-06,1285.449951,1248,35.0,0.00,NaN,Winter,2006,1,1
4,2006-01-09,1290.150024,896,50.0,0.00,NaN,Winter,2006,2,1
...,...,...,...,...,...,...,...,...,...,...
2512,2015-12-24,2060.989990,311,63.0,0.02,Fog-Rain,Winter,2015,52,12
2513,2015-12-28,2056.500000,659,42.0,0.05,Rain-Snow,Winter,2015,53,12
2514,2015-12-29,2078.360107,810,42.0,0.66,Rain-Snow,Winter,2015,53,12
2515,2015-12-30,2063.360107,645,46.0,0.37,Rain,Winter,2015,53,12


In [5]:
app.layout = html.Div(
    children=[
        html.H1(children="Dash Test"),
        html.Div(
            children="""
                     Dash: A web application framework for Python.
                     """
        ),
        html.Div(
            id="radio-items",
            children=dcc.RadioItems(
                id="radio-test",
                options=[
                    {"label": "New York City", "value": "NYC"},
                    {"label": "Montréal", "value": "MTL"},
                ],
                value="MTL",
            ),
        ),
        # dcc.Graph(id="example-graph"),
        html.H3(id="graph-title", children="This is the graph title"),
        dcc.Graph(id="weekly-by-year"),
        dcc.Slider(
            id="years-slider",
            min=weekly["Year"].min(),
            max=weekly["Year"].max(),
            value=weekly["Year"].min(),
            marks={str(year): {"label": str(year)} for year in weekly["Year"].unique()},
        ),
        html.Br(),
    ]
)
@app.callback(Output("weekly-by-year", "figure"), [Input("years-slider", "value")])
def update_weekly_app(year):
    return update_weekly(year)

NameError: name 'weekly' is not defined

In [ ]:
def update_weekly(year, value):
    weekly_by_year = weekly.loc[
        weekly["Year"] == year,
        ["Week", "Close", "Num Arrests", "Month", "Mean.TemperatureF"],
    ]
    fig = go.Figure()
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(
            x=weekly_by_year["Week"],
            y=weekly_by_year["Num Arrests"],
            name="Number of Crimes",
            mode="markers",
            yaxis="y1",
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=weekly_by_year["Week"],
            y=weekly_by_year[value],
            name="S&P500 Close Price",
            yaxis="y2",
        ),
        secondary_y=True,
    )
    fig.update_xaxes(
        ticktext=[
            "January",
            "March",
            "June",
            "August",
            "October",
            "December",
        ],  # 1, 3, 5, 7, 9, 11
        tickvals=[1, 10, 20, 30, 40, 50],
    )
    fig.update_layout(
        xaxis={"zeroline": False},
        title=year,
        transition={"duration": 1000},
        yaxis1={"title": "Number of Arrests"},
        yaxis2={"title": "S&P"},
    )
    print(year)
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)